<a href="https://colab.research.google.com/github/ZeleOeO/Artificial-Intelligence-Projects/blob/main/News_Categoriser_V2_ipnyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(ngram_range = (1,4))
vec.fit(["Thor Hathodawal is looking for a job"])
vec.vocabulary_

{'thor': 14,
 'hathodawal': 2,
 'is': 6,
 'looking': 11,
 'for': 0,
 'job': 10,
 'thor hathodawal': 15,
 'hathodawal is': 3,
 'is looking': 7,
 'looking for': 12,
 'for job': 1,
 'thor hathodawal is': 16,
 'hathodawal is looking': 4,
 'is looking for': 8,
 'looking for job': 13,
 'thor hathodawal is looking': 17,
 'hathodawal is looking for': 5,
 'is looking for job': 9}

In [ ]:
corpus = [
     "Thor ate the pizza",
     "Loki is tall",
     "Loki is eating pizza"
 ] 

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def preprocess(text):
  doc = nlp(text)

  filtered_tokens=[]

  for token in doc:
    if token.is_stop or token.is_punct:
      continue
    filtered_tokens.append(token.lemma_)
  
  return " ".join(filtered_tokens)

preprocess("Loki is eating pizza")

'Loki eat pizza'

In [ ]:
corpus_processed = [preprocess(i) for i in corpus]
corpus_processed

['thor eat pizza', 'Loki tall', 'Loki eat pizza']

In [ ]:
vec = CountVectorizer(ngram_range=(1,2))
vec.fit(corpus_processed)
vec.vocabulary_

{'thor': 7,
 'eat': 0,
 'pizza': 5,
 'thor eat': 8,
 'eat pizza': 1,
 'loki': 2,
 'tall': 6,
 'loki tall': 4,
 'loki eat': 3}

In [ ]:
vec.transform(corpus_processed).toarray()

array([[1, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 0, 1, 0, 0],
       [1, 1, 1, 1, 0, 1, 0, 0, 0]])

In [ ]:
import pandas as pd

url="https://github.com/codebasics/nlp-tutorials/blob/main/11_bag_of_n_grams/news_dataset.json?raw=true"
df = pd.read_json(url)
df.shape

(12695, 2)

In [ ]:
df.category.value_counts()

BUSINESS    4254
SPORTS      4167
CRIME       2893
SCIENCE     1381
Name: category, dtype: int64

In [ ]:
min_samples = 1381

df[df.category=="BUSINESS"] = df[df.category=="BUSINESS"].sample(min_samples, random_state=2023)
df[df.category=="SPORTS"] = df[df.category=="SPORTS"].sample(min_samples, random_state=2023)
df[df.category=="CRIME"] = df[df.category=="CRIME"].sample(min_samples, random_state=2023)
df = df.dropna()
df.category.value_counts()

SCIENCE     1381
BUSINESS    1381
SPORTS      1381
CRIME       1381
Name: category, dtype: int64

In [ ]:
df["category_num"] = df.category.map({
    "BUSINESS":0,
    "SPORTS":1,
    "CRIME":2,
    "SCIENCE":3
})

df

,text,category,category_num
0,Watching Schrödinger's Cat Die University of C...,SCIENCE,3
1,WATCH: Freaky Vortex Opens Up In Flooded Lake,SCIENCE,3
9,Here's Why Coffee Makes You Have To Poop It hi...,SCIENCE,3
11,Bad News For Trump Is Good News For The Stock ...,BUSINESS,0
19,ESPN's Interruption Of Dabo Swinney Interview ...,SPORTS,1
...,...,...,...
12684,Woman Shot By Former Boyfriend At Chicago Nord...,CRIME,2
12687,"Get Even I was in debt. Serious debt. The ""how...",BUSINESS,0
12689,5-Year-Old Girl Abandoned At NYC Bus Terminal;...,CRIME,2
12691,This Minivan-Sized Sea Sponge Is Thought To Be...,SCIENCE,3


In [ ]:
df["pre_processed_text"] = df.text.apply(preprocess)

In [ ]:
df.head()

,text,category,category_num,pre_processed_text
0,Watching Schrödinger's Cat Die University of C...,SCIENCE,3,watch Schrödinger Cat Die University Californi...
1,WATCH: Freaky Vortex Opens Up In Flooded Lake,SCIENCE,3,WATCH Freaky vortex open Flooded Lake
9,Here's Why Coffee Makes You Have To Poop It hi...,SCIENCE,3,Coffee make Poop hit minute cup
11,Bad News For Trump Is Good News For The Stock ...,BUSINESS,0,Bad News Trump good News Stock Market slight b...
19,ESPN's Interruption Of Dabo Swinney Interview ...,SPORTS,1,ESPN Interruption Dabo Swinney Interview Frigg...


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test =  train_test_split(
    df.pre_processed_text,
    df.category_num,
    test_size = 0.2,
    random_state=2023,
    stratify = df.category_num
)

In [ ]:
y_train.value_counts()

0    1105
1    1105
2    1105
3    1104
Name: category_num, dtype: int64

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
vec2=CountVectorizer()
model=MultinomialNB()

clf = Pipeline([
    ("vectorizer_bow", vec2),
    ("Multi NB", model),
])

clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86       276
           1       0.90      0.86      0.88       276
           2       0.85      0.92      0.88       276
           3       0.92      0.82      0.87       277

    accuracy                           0.87      1105
   macro avg       0.88      0.87      0.87      1105
weighted avg       0.88      0.87      0.87      1105



In [44]:
news = [
    "Two men shot and killed at Johannesburg",
    "Stock Market Crashing",
    "Arsenal to beat Tottenham at Europa League",
    "How Peter Obi spent Easter in detention in London",
    "Fuel price soars by 81% despite N6tn subsidy payment",
    "National Assembly proposes jail terms for Ponzi scheme promoters",
    "How gunmen abducted Kogi monarch, killed female domestic worker - Police",
    "PDP conducts Bayelsa gov election primaries",
    "INEC to reset 2,500 BVAS for supplementary polls",
    "Bidding process for Man United enters third round ",
    "UCL: Napoli confident of victory without Osimhen - Di Lorenzo ",
    "Victory as thirteen men killed on bridge",
    "Videos of gold nanoparticles snapping together show how some crystals grow ",
    "This sea cucumber shoots sticky tubes out of its butt. Its genes hint at how",
    "Dense crowds of pedestrians shift into surprisingly orderly lines. Math explains why "
]

news_count = vec2.transform([preprocess(i) for i in news])

for index, i in enumerate(model.predict(news_count)):
  if i == 0:
    print(news[index], " |   BUSINESS NEWS", "\t\t\t\t", i)
  elif i == 1:
    print(news[index], " |   SPORTS NEWS", "\t\t\t\t", i)
  elif i == 2:
    print(news[index], " |   CRIME NEWS", "\t\t\t\t", i)
  elif i == 3:
    print(news[index], " |   SCIENCE NEWS", "\t\t\t\t", i)

Two men shot and killed at Johannesburg  |   CRIME NEWS 				 2
Stock Market Crashing  |   BUSINESS NEWS 				 0
Arsenal to beat Tottenham at Europa League  |   SPORTS NEWS 				 1
How Peter Obi spent Easter in detention in London  |   CRIME NEWS 				 2
Fuel price soars by 81% despite N6tn subsidy payment  |   BUSINESS NEWS 				 0
National Assembly proposes jail terms for Ponzi scheme promoters  |   BUSINESS NEWS 				 0
How gunmen abducted Kogi monarch, killed female domestic worker - Police  |   CRIME NEWS 				 2
PDP conducts Bayelsa gov election primaries  |   BUSINESS NEWS 				 0
INEC to reset 2,500 BVAS for supplementary polls  |   SCIENCE NEWS 				 3
Bidding process for Man United enters third round   |   CRIME NEWS 				 2
UCL: Napoli confident of victory without Osimhen - Di Lorenzo   |   SPORTS NEWS 				 1
Victory as thirteen men killed on bridge  |   CRIME NEWS 				 2
Videos of gold nanoparticles snapping together show how some crystals grow   |   SCIENCE NEWS 				 3
This sea cucu